# Exploitation Zone — Text Processing

This notebook handles the **text processing** step for the Exploitation Zone of our data pipeline.  
Its primary goal is to:

1. **Load documents** from the Trusted Zone
2. **Generate embeddings** from the documents in the Trusted Zone
3. **Store the embeddings** in ChromaDB



## 1. Setup and Configuration


In [ ]:
import os, json, gzip
from datetime import datetime, timezone
from typing import Iterable, List

import boto3
from botocore.config import Config
from dotenv import load_dotenv

import chromadb
from chromadb import PersistentClient
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction, OpenCLIPEmbeddingFunction

load_dotenv()

MINIO_USER     = os.getenv("MINIO_USER")
MINIO_PASSWORD = os.getenv("MINIO_PASSWORD")
MINIO_ENDPOINT = os.getenv("MINIO_ENDPOINT")

TRUST_BUCKET         = "trusted-zone"
TRUST_RECIPES_PREFIX = "documents"

PERSIST_DIR = os.getenv("CHROMA_PERSIST_DIR")

BATCH = 256


session = boto3.session.Session(
    aws_access_key_id=MINIO_USER,
    aws_secret_access_key=MINIO_PASSWORD,
    region_name="us-east-1",
)
s3 = session.client(
    "s3",
    endpoint_url=MINIO_ENDPOINT,
    config=Config(signature_version="s3v4", s3={"addressing_style": "path"}),
)

chroma_client = PersistentClient(path=PERSIST_DIR)
ef_text = SentenceTransformerEmbeddingFunction(model_name="Qwen/Qwen3-Embedding-0.6B")
ef_multi = OpenCLIPEmbeddingFunction()

docu_name = "trusted_zone_documents"
multi_name = "trusted_zone_multimodal"

recipes_col = chroma_client.get_or_create_collection(
    name=docu_name,
    embedding_function=ef_text,
    metadata={"modality": "text", "model": "Qwen/Qwen3-Embedding-0.6B", "source": "minio"},
)

multi_col = chroma_client.get_or_create_collection(
    name=multi_name,
    embedding_function=ef_multi,
    metadata={"modality": "multimodal", "model": "OpenCLIP", "source": "minio"}
)
print("✅ Chroma DB directory:", PERSIST_DIR)

## 2. Helper Functions


In [ ]:
def utc_ts() -> str:
    return datetime.now(timezone.utc).strftime("%Y-%m-%dT%H-%M-%SZ")


def list_all_recipe_keys(bucket: str, prefix: str) -> List[str]:
    keys, token = [], None
    exts = (".jsonl")
    while True:
        kwargs = {"Bucket": bucket, "Prefix": prefix, "MaxKeys": 1000}
        if token: kwargs["ContinuationToken"] = token
        resp = s3.list_objects_v2(**kwargs)
        for obj in resp.get("Contents", []):
            k = obj["Key"]
            if k.lower().endswith(exts):
                keys.append(k)
                print(f"Found recipe file: {k}")
        token = resp.get("NextContinuationToken")
        if not token:
            break
    return keys


def stream_objects_lines(bucket: str, key: str) -> Iterable[str]:
    obj = s3.get_object(Bucket=bucket, Key=key)
    raw = obj["Body"].read()
    if key.lower().endswith(".gz"):
        raw = gzip.decompress(raw)

    text = raw.decode("utf-8", errors="replace")
    if key.lower().endswith(".jsonl"):
        for line in text.splitlines():
            line = line.strip()
            if line:
                yield line


def normalize_recipe_json(rec: dict):
    rid = str(rec.get("id") or "").strip()
    if not rid:
        return None

    title = (rec.get("title_text_clean")
             or rec.get("title__from_recipes_with_nutritional_info")
             or rec.get("title__from_layer1")
             or rec.get("title_text_raw")
             or "").strip()

    ing_clean = rec.get("ingredients_text_clean")
    if not ing_clean:
        ing_lists = (rec.get("ingredients__from_recipes_with_nutritional_info")
                     or rec.get("ingredients__from_layer1")
                     or rec.get("ingredients__from_det_ingrs")
                     or [])
        ing_clean = " ".join(x.get("text","").strip() for x in ing_lists if isinstance(x, dict))

    instr_clean = rec.get("instructions_text_clean")
    if not instr_clean:
        instr_lists = (rec.get("instructions__from_recipes_with_nutritional_info")
                       or rec.get("instructions__from_layer1")
                       or [])
        instr_clean = " ".join(x.get("text","").strip() for x in instr_lists if isinstance(x, dict))

    parts = []
    if title: parts.append(f"title: {title}")
    if ing_clean: parts.append(f"ingredients: {ing_clean}")
    if instr_clean: parts.append(f"instructions: {instr_clean}")
    document = "\n".join(parts).strip()
    if not document:
        return None

    lights = rec.get("fsa_lights_per100g__from_recipes_with_nutritional_info") or {}
    
    # Build metadata and filter out None values
    meta = {
        "id": rid,
        "title": title,
        "has_nutrition": bool(rec.get("nutrition_normalized")),
        "source_bucket": "trusted-zone",
    }
    
    # Only add FSA values if they're not None
    if lights.get("fat") is not None:
        meta["fsa_fat"] = lights.get("fat")
    if lights.get("salt") is not None:
        meta["fsa_salt"] = lights.get("salt")
    if lights.get("saturates") is not None:
        meta["fsa_saturates"] = lights.get("saturates")
    if lights.get("sugars") is not None:
        meta["fsa_sugars"] = lights.get("sugars")

    return {"id": rid, "document": document, "metadata": meta}

This block defines utility functions for listing, streaming, and normalizing recipe data stored as JSONL objects in S3 buckets.

`utc_ts` generates a UTC timestamp in ISO-like format, used for consistent logging or versioning.
`list_all_recipe_keys` iterates through S3 objects under a given bucket and prefix, paginating with continuation tokens and collecting all keys ending in `.jsonl`, printing each recipe file found.

`stream_objects_lines` retrieves an object’s raw bytes from S3, automatically decompressing `.gz` files when needed. It decodes the text as UTF-8 and yields non-empty lines from `.jsonl` files, allowing recipes to be processed line by line without loading entire datasets into memory.

`normalize_recipe_json` cleans and standardizes raw recipe dictionaries. It extracts core fields such as `id`, `title`, `ingredients`, and `instructions` from various possible source attributes, consolidating them into a single coherent text document. The function also constructs a metadata dictionary containing identifiers, nutrition flags, and FSA nutritional “traffic light” values (fat, salt, saturates, sugars), tagging the source bucket as `"trusted-zone"`.

Together, these functions support scalable ingestion and normalization of recipe data, preparing it for indexing, analysis, or embedding downstream.


## 3. Store rececipes ingredients, instructions and metadata as embeddings in chormaDB

In [ ]:
def ingest_recipes(bucket: str, prefix: str, batch: int = BATCH, collection = None):
    buf_ids, buf_docs, buf_meta = [], [], []
    n_ok, n_bad = 0, 0

    keys = list_all_recipe_keys(bucket, prefix)
    for key in keys:
        for payload in stream_objects_lines(bucket, key):
            try:
                data = json.loads(payload)
                if isinstance(data, dict):
                    items = [data]
                elif isinstance(data, list):
                    items = data
                else:
                    raise ValueError("JSON root is not a dict or list")

                for rec in items:
                    if not isinstance(rec, dict):
                        n_bad += 1
                        continue
                    pack = normalize_recipe_json(rec)
                    if not pack:
                        n_bad += 1
                        continue

                    buf_ids.append(pack["id"])
                    buf_docs.append(pack["document"])
                    buf_meta.append({"type": "text"} | pack["metadata"])
                    n_ok += 1

                    if len(buf_ids) >= batch:
                        collection.add(ids=buf_ids, documents=buf_docs, metadatas=buf_meta)
                        buf_ids, buf_docs, buf_meta = [], [], []

            except Exception as e:
                n_bad += 1

    if buf_ids:
        collection.add(ids=buf_ids, documents=buf_docs, metadatas=buf_meta)

    print(f"✅ Ingestion completed. OK: {n_ok}  |  discarded: {n_bad}")
#Have to separate these calls to avoid issues with caching
ingest_recipes(TRUST_BUCKET, TRUST_RECIPES_PREFIX, batch=BATCH, collection=recipes_col)
ingest_recipes(TRUST_BUCKET, TRUST_RECIPES_PREFIX, batch=BATCH, collection=multi_col)

In [ ]:
query = "honey sriracha chicken wings"
res = recipes_col.query(
    query_texts=[query],
    n_results=1,
    include=["metadatas", "distances", "documents"],
)
print("🔎 Query:", query)
print(json.dumps(res, indent=2, ensure_ascii=False))

In [ ]:
def show_recipe(res):
    best_doc = res["documents"][0][0]
    meta = res["metadatas"][0][0]
    dist = float(res["distances"][0][0])
    sim = 1.0 / (1.0 + dist)

    fsa_badge = {
        "green": "🟢",
        "orange": "🟠",
        "red": "🔴",
        None: "⚪",
    }
    print("🏆 Closest Match")
    print("=" * 50)
    print(f"ID: {meta.get('id')}")
    print(f"Title: {str(meta.get('title','')).strip().capitalize()}")
    print(f"Distance: {dist:.3f}  |  Similarity: {sim:.3f}")
    print("FSA Lights → "
          f"fat:{fsa_badge.get(meta.get('fsa_fat'))}  "
          f"salt:{fsa_badge.get(meta.get('fsa_salt'))}  "
          f"saturates:{fsa_badge.get(meta.get('fsa_saturates'))}  "
          f"sugars:{fsa_badge.get(meta.get('fsa_sugars'))}")
    print("\n--- Recipe ---")

    for line in best_doc.split("\n"):
        if line.startswith("title:"):
            print(f"\n🧁 {line.replace('title:', '').strip().capitalize()}")
        elif line.startswith("ingredients:"):
            print("\n🥣 Ingredients:")
            ings = line.replace("ingredients:", "").strip()
            print("   " + ings.replace("  ", " "))
        elif line.startswith("instructions:"):
            print("\n👩‍🍳 Instructions:")
            instr = line.replace("instructions:", "").strip()
            print("   " + instr.replace("  ", " "))
    print("=" * 50)


show_recipe(res)



This block ingests recipe data from the trusted S3 bucket, normalizes it, and indexes it into a ChromaDB collection for semantic search.

`ingest_recipes` scans all `.jsonl` recipe files under the specified prefix, streaming each line directly from S3. It parses each JSON payload, cleans and standardizes its structure through `normalize_recipe_json`, and batches valid records into the vector database. Invalid or incomplete entries are counted and skipped, while successful ones are tracked for reporting.

After ingestion, a sample query demonstrates text-to-recipe retrieval using semantic similarity. The `show_recipe` function formats the best match, displaying its title, FSA nutrition badges, and cleaned ingredients and instructions in a readable format.

Together, these routines implement an end-to-end pipeline for structured recipe ingestion, embedding, and retrieval from the trusted data zone.